In [8]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd

In [ ]:
from ultralytics import YOLO

In [ ]:
Directory = r"F:\Work\SOCAAutomation\Dataset\Images\09072024_1_1"
FileName = "2024_07_09_11_30_174.jpg"
raw_img = cv.imread(Directory + "/" + FileName)
img = cv.cvtColor(raw_img, cv.COLOR_BGR2GRAY)
img = raw_img

In [2]:
model = YOLO('best.pt', task='segment')

In [ ]:
model = YOLO('best.pt')
results = model(img, conf=0.3, save=True, show_labels=False, retina_masks=True)

**Pre and Post processing time**
- reduced from 5 seconds to ms by resizing the image before applying to the model

# Video Inferencing

In [ ]:
from ultralytics.utils.plotting import Annotator, colors

In [52]:
img_resize = cv.resize(frame, (width, height), interpolation = cv.INTER_LINEAR)
results = model(img_resize, conf=0.5)
if results[0].masks is not None:
    mask_img = np.any(np.array(results[0].masks[:].data), axis=0)
    mask_img_3 = np.stack([mask_img, np.zeros(mask_img.shape), np.zeros(mask_img.shape)], axis=2)
    temp_image = cv.addWeighted(img_resize, alpha , (mask_img_3 * 255).astype('uint8'), 1-alpha, 0)        
    im_out = cv.warpPerspective(mask_img_3, h, (im_layout.shape[1],im_layout.shape[0]))
    layout_mask = cv.addWeighted(im_layout, 0.3 , (im_out * 255).astype('uint8'), 1-0.3, 0)
layout_mask = cv.addWeighted(im_layout, 0.3 , (im_out * 255).astype('uint8'), 1-0.3, 0)
im_show(layout_mask)

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [49]:
def im_show(img):
    cv.imshow('1', img)
    cv.waitKey()

In [3]:
# Camera Perspective View to Top View Transformation
def camera_angle2top_view():
    points = pd.read_csv('Points.csv')
    ref_pts = np.array([points.iloc[i,:2] for i in range(1, len(points))], dtype = 'uint16')
    dst_pts = np.array([points.iloc[i,2:] for i in range(1, len(points))], dtype = 'uint16') * 2
    h, status = cv.findHomography(ref_pts, dst_pts)
    return h

In [36]:
# def InferenceImage(frame, model, width, height, alpha, im_layout, h):
#     img_resize = cv.resize(frame, (width, height), interpolation = cv.INTER_LINEAR)
#     results = model(img_resize, conf=0.5)
#     if results[0].masks is not None:
#         mask_img = np.any(np.array(results[0].masks[:].data), axis=0)
#         mask_img_3 = np.stack([mask_img, np.zeros(mask_img.shape), np.zeros(mask_img.shape)], axis=2)
#         temp_image = cv.addWeighted(img_resize, alpha , (mask_img_3 * 255).astype('uint8'), 1-alpha, 0)        
#         im_out = cv.warpPerspective(mask_img_3, h, (im_layout.shape[1],im_layout.shape[0]))
#         layout_mask = cv.addWeighted(im_layout, 0.3 , (im_out * 255).astype('uint8'), 1-0.3, 0)
# #         mask_inv = np.logical_not(np.stack([mask_img, mask_img, mask_img], axis=2))
# #         temp_image = np.zeros(orig_img.shape)
# #         temp_image = mask_inv * orig_img
# #         temp_image += (mask_img_3 * 255).astype('uint8')
#     else:
#         temp_image = img_resize
#         layout_mask = im_layout
#     return temp_image, layout_mask

In [67]:
count_frame = 0
fps = 1
alpha = 0.8
Directory = r"F:\Work\SOCAAutomation\Dataset\Images\09072024_1_1"

width = height = 640
Files = os.listdir(Directory)
Files = ['2024_07_09_11_30_114.jpg', '2024_07_09_11_30_115.jpg', '2024_07_09_11_30_116.jpg', '2024_07_09_11_30_117.jpg', '2024_07_09_11_30_118.jpg']
im_ref = cv.imread('ReferenceImage.jpg')
#im_layout = cv.imread('Layout.jpg')    

In [68]:
model = YOLO('best.pt')
out = cv.VideoWriter("inst_segment.avi", cv.VideoWriter_fourcc(*"MJPG"), fps, (width, height))
out2 = cv.VideoWriter("inst_seg_layout.avi", cv.VideoWriter_fourcc(*"MJPG"), fps, (im_layout.shape[1], im_layout.shape[0]))
video = cv.VideoCapture(r"F:\Work\SOCAAutomation\Dataset\NA_DISOPSAL_SOCA_2024\09072024\AXIS 213 - 10.1.5.173 2024-07-09_11_30_00_000.asf")
dur = [[3,0], [3,45]] #[mins,secs]
startFrame = (dur[0][0] * 60 + dur[0][1] ) * 15
endFrame = (dur[1][0] * 60 + dur[1][1] ) * 15

In [69]:
count_frame = 0
h = camera_angle2top_view()
while True:
    ret, frame = video.read()
    if ret:
        if count_frame >= 0 and count_frame < 4:
            frame = cv.imread(Directory + "/" + Files[count_frame])
            img_resize = cv.resize(frame, (width, height), interpolation = cv.INTER_LINEAR)
            results = model(img_resize, conf=0.5)
            if results[0].masks is not None:
                mask_img = np.any(np.array(results[0].masks[:].data), axis=0)
                mask_img_3 = np.stack([np.zeros(mask_img.shape), np.zeros(mask_img.shape), mask_img], axis=2)
                temp_image = cv.addWeighted(img_resize, alpha , (mask_img_3 * 255).astype('uint8'), 1-alpha, 0)        
                im_out = cv.warpPerspective(mask_img_3, h, (im_layout.shape[1],im_layout.shape[0]))
                layout_mask = cv.addWeighted(im_layout, 0.3 , (im_out * 255).astype('uint8'), 1-0.3, 0)
            else:
                temp_image = img_resize
                layout_mask = im_layout
                
            out.write(temp_image)
            out2.write(layout_mask)
            count_frame += 1
    else:
        break
out.release()
out2.release()
video.release()


0: 640x640 4 Fire-Smoke-Sodium-Trays, 4071.7ms
Speed: 14.0ms preprocess, 4071.7ms inference, 16.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 Fire-Smoke-Sodium-Trays, 4060.7ms
Speed: 7.0ms preprocess, 4060.7ms inference, 12.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 4 Fire-Smoke-Sodium-Trays, 4351.9ms
Speed: 7.0ms preprocess, 4351.9ms inference, 25.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 4 Fire-Smoke-Sodium-Trays, 4253.8ms
Speed: 7.0ms preprocess, 4253.8ms inference, 17.0ms postprocess per image at shape (1, 3, 640, 640)


In [65]:
mask_img_3 = np.stack([np.zeros(mask_img.shape), np.zeros(mask_img.shape), mask_img], axis=2)
temp_image = cv.addWeighted(img_resize, alpha , (mask_img_3 * 255).astype('uint8'), 1-alpha, 0)        
im_out = cv.warpPerspective(mask_img_3, h, (im_layout.shape[1],im_layout.shape[0]))
layout_mask = cv.addWeighted(im_layout, 0.3 , (im_out * 255).astype('uint8'), 1-0.3, 0)

In [73]:
test = np.zeros((640,640,3))
test= layout_mask

In [76]:
pad_arr = np.pad(layout_mask, (640-layout_mask.shape[0],640-layout_mask.shape[1], 3), 0) 

ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (3,)  and requested shape (3,2)

In [77]:
layout_mask.shape

(578, 478, 3)

In [74]:
im_show(test)

In [13]:
out.release()
video.release()

In [ ]:
cv.imshow('window', temp_image)
cv.waitKey()

In [ ]:
alpha = 0.8
dst = cv.addWeighted(im1, alpha , im2, 1-alpha, 0) 
plt.imshow(im2)

In [ ]:
alpha = 0.6
im1 = cv.imread(r'F:\Work\SOCAAutomation\Dataset\Images\09072024_1_1\2024_07_09_11_30_32.jpg')
im1 = cv.resize(im1, (640, 640), interpolation = cv.INTER_LINEAR)
#results = model(im1, conf=0.5) #, save=True, show_labels=False, retina_masks=True)
mask_img = np.any(np.array(results[0].masks[:].data), axis=0)
orig_img = im1 #results[0].orig_img
mask_img_3 = np.stack([mask_img, np.zeros(mask_img.shape), np.zeros(mask_img.shape)], axis=2)

im2 = np.array(mask_img_3 * 255, dtype = 'uint8')

dst = cv.addWeighted(im1, alpha , im2, 1-alpha, 0) 
# mask_inv = np.logical_not(np.stack([mask_img, mask_img, mask_img], axis=2))
# temp_image = np.zeros(orig_img.shape)
# temp_image = mask_inv * orig_img
# temp_image += (mask_img_3 * 255).astype('uint8')


In [ ]:
plt.imshow(mask_img_3)

In [ ]:
plt.imshow(dst)

In [ ]:
frame = cv.imread(r'F:\Work\SOCAAutomation\Dataset\Images\09072024_1_1\2024_07_09_11_30_344.jpg')
img_resize = cv.resize(frame, (640, 640), interpolation = cv.INTER_LINEAR)
cv.imwrite('ReferenceImage.jpg', img_resize)

# Transform Model to Onnx

In [ ]:
model.export(format="onnx")  # creates 'yolov8n_openvino_model/

In [ ]:
onnx_model = YOLO("best.onnx", task='segment')

In [ ]:
res_onnx = onnx_model(img, conf=0.3)

## ONNX Model Optimizations

### F32 to F16

In [ ]:
import onnx
from onnxconverter_common import float16

In [ ]:
onnx_model = onnx.load("best.onnx")
onnx_model_fp16 = float16.convert_float_to_float16(onnx_model)
onnx.save(onnx_model_fp16, "model_fp16.onnx")

In [ ]:
onnx_model = YOLO("model_fp16.onnx", task='segment')

In [ ]:
res_onnx_fp16 = onnx_model(img, conf=0.3)

In [ ]:
img.dtype

# Transform Model to OpenVino

In [ ]:
model.export(format="openvino")  # creates 'yolov8n_openvino_model/

In [ ]:
ov_model = YOLO("yolov8n_openvino_model/")

# Run inference
results = ov_model("https://ultralytics.com/images/bus.jpg")